# Introduction

TileDB-VCF offers several interfaces to create and/or query TileDB arrays containing genomic variant data. Here we'll utilize the command-line interface (CLI) and walk through the process of loading raw VCF data into TileDB and then performing a few different types of exports.

***Hint: By the way, you can launch an interactive version of this tutoral using [TileDB Cloud Notebooks](https://console.tiledb.com/notebooks). Simply start a session with the Genomics & Geospatial image and use the integrated file browser to open the notebook: `examples/genomics/01-vcf-cli.ipynb`.***


# Create a Dataset

The process of creating a new VCF dataset involves 3 phases: `create`, `register`, and `store`. Detailed information about each phase is provided in the [ingestion algorithm](https://docs.tiledb.com/genomics/advanced/ingestion-algorithm) section.


We'll start with a small example using 3 synthetic VCF files available in `data/vcfs`:

In [ ]:
tree data

Index files are required for ingestion. If your VCF/BCF files have not been indexed you can use [`bcftools`](https://samtools.github.io/bcftools/bcftools.html) to do so:

```shell
for f in data/vcfs/*.vcf.gz; do bcftools index -c $f; done
```

The first step is to **create** an empty dataset. Let's save the dataset in a new directory called `small_dataset`:

In [ ]:
tiledbvcf create --uri small_dataset

The next step is to **register** the samples that will be ingested using either a text file that contains the location of each VCF or by directly passing a list of VCF files to the command.

In [ ]:
tiledbvcf register --uri small_dataset data/vcfs/G*.vcf.gz

You can always determine which samples have been registered with a dataset using the `list` command:

In [ ]:
tiledbvcf list --uri small_dataset

The final step is to actually ingest the VCF files and **store** their content within `small_dataset`.

In [ ]:
tiledbvcf store --uri small_dataset data/vcfs/G*.vcf.gz

That's it! Let's verify evertyhing went okay using the `stat` command to provide high-level statistics about our dataset including the number of samples it contains and the variant attributes it includes.

In [ ]:
tiledbvcf stat --uri small_dataset

At this point you have succesfully created and populated a TileDB VCF dataset. By default *all* metadata recorded in the VCF data lines are ingested but you can override this behavior with the `--attributes` flag, which allows you to specify the subset of fields you want to include.



# Incremental Updates

One key advantage to using TileDB as the data store for your genomic variant data is the ability to efficiently add new samples as they become available. Furthermore, TileDB's native cloud features make it possible to ingest samples directly from remote locations. Here, we'll register and ingest the following additional samples, which are located on [AWS S3](https://aws.amazon.com/s3/):

In [ ]:
cat data/s3-bcf-samples.txt

*Note: Samples in this second batch are stored as `BCF` files which are also supported by TileDB-VCF.*

This process is identical to the steps perfomed above, the only changes needed to our code involve setting `--scratch-mb` to allocate some temporary space for downloading the files and providing the URLs for the remote files. In this case, we'll simply pass the `s3-bcf-samples.txt` file, which includes a list of the BCF files we want to register and ingest.

In [ ]:
tiledbvcf register \
  --uri small_dataset \
  --scratch-mb 1 \
  --samples-file data/s3-bcf-samples.txt

*Note: When ingesting samples from S3, you must configure enough scratch space to hold at least 20 samples. In general, you need 2 &times; the sample dimension's `tile_extent` (which by default is 10). You can read more about the data model [here](https://docs.tiledb.com/genomics/advanced/data-mode)).*

You can add the `--verbose` flag to print out more information during the `store` phase.

In [ ]:
tiledbvcf store \
  --uri small_dataset \
  --scratch-mb 10 \
  --samples-file data/s3-bcf-samples.txt \
  --verbose

Again, let's run the `stat` command to verify our dataset now includes 10 samples.

In [ ]:
tiledbvcf stat --uri small_dataset

Here, you've updated an existing dataset by adding new *BCF files* located on a *remote* server. Because TileDB is designed to be updatable, this process happens efficiently and without ever touching the previously ingested data—avoiding computationally expensive operations like regenerating combined VCF files.

# Querying and Exporting

Now that you've created a TileDB-VCF dataset, you can begin to query the variant data for the 10 ingested samples. As mentioned earlier, several APIs are available for accessing the data. See the [Python usage section](https://docs.tiledb.com/genomics/usage/python) for examples using the `tiledbvcf` Python module. Here, we'll continue working with the CLI, focusing now on the `export` command, which can produce 3 different types of outputs for any given query:

1. VCF (or BCF), producing one `.vcf` output file per exported sample
2. Tabular, producing a single tab-separated value (TSV) text file containing all intersecting records across the exported samples
3. Count-only, outputs a count of the total number of intersecting records (no output file is produced)

The following examples provide a high-level overview of the CLI's export functionality; see `tiledbvcf export --help` or the [TileDB-VCF website](https://docs.tiledb.com/genomics/apis/cli) for comprehensive documentation.

## Export VCF/BCFs

In this example we will export several genomic regions from `small_dataset` created above:

In [ ]:
mkdir -p data/exported-subsets

tiledbvcf export \
  --uri small_dataset \
  --regions 1:1-50000,2:1-50000,3:1-50000 \
  --sample-names G1,G2,G3 \
  --output-dir data/exported-subsets \
  --verbose

This produced the three `bcf` files shown below, each of which contains the records intersecting the specified regions for the corresponding sample.

In [ ]:
tree data/exported-subsets

You can use `bcftools` to examine any of the exported `bcf` files:

In [ ]:
bcftools view --no-header data/exported-subsets/G1.bcf

***Note: If a sample does not contain any intersecting records an output file is still created but it will include 0 records.***

In order to export *all* records, simply omit the `--regions` argument. For example, the following recovers the original BCFs for samples `G4` and `G5`:

In [ ]:
mkdir -p data/exported-full

tiledbvcf export \
  --uri small_dataset \
  --sample-names G4,G5 \
  --output-dir data/exported-full \
  --verbose

This will output two BCFs for the corresponding samples that include the complete set of information from the original files. Note, while these exports are *lossless* in terms of the actual data stored, they may not be *identical* to the original files. Foe example, exported BCF/VCFs will always include an `END` position field even if one was not present in the ingested files. Furthermore, the ordering of fields within the `INFO` and `FORMAT` columns may also differ. 

## Export to TSV

Next we'll look at exporting data from `small_dataset` in *tabular* format, which makes it convenient to load into other tools for downstream analyses. The command arguments are largely the same as for exporting to VCF/BCF, you just need to change the output type to `t` (short for TSV) and specify which VCF fields should be included as columns in the output table. The standard field names are:

- `SAMPLE` (always included)
- `ID`
- `REF`
- `ALT`
- `QUAL`
- `CHR`
- `POS`
- `FILTER`

Fields within the `FORMAT` and `INFO` columns are accessed using special prefixes, `S:` and `I:`, respectively. This notation is used below to include genotype information from the `FORMAT:GT` field:

In [ ]:
tiledbvcf export \
    --uri small_dataset \
    -Ot --tsv-fields CHR,POS,I:END,REF,S:GT \
    -s G1,G2,G3,G4 \
    --regions 1:1-50000 \
    --verbose

The results are output in a long form table where each row represents a single variant that overlapped one of the specified query regions for a particular sample. 

There's one more special prefix worth pointing out: `Q:` (for *query*), which allows you to include columns containing the start (`Q:POS`) and end (`Q:END`) positions for the query region in which the variant is located. Here we perform the same query across a few more regions, add the query regions columns, and save the output to `data/exported-regions.tsv`.

In [ ]:
tiledbvcf export \
    --uri small_dataset \
    -Ot --tsv-fields Q:POS,Q:END,CHR,POS,I:END,REF,S:GT \
    -s G1,G2,G3,G4 \
    --regions 1:1-50000,2:60000-100000,3:110000-160000 \
    --output-path data/exported-regions.tsv

cat data/exported-regions.tsv

The first row shows that sample `G3` contains a variant stretching from 11,758–92,302 on chromosome 1 overlaps the first query region.

# Session

In [ ]:
echo "Updated: $(date)"

In [ ]:
tiledbvcf --version